<center><h1>socketserver</h1></center>

# 1 Introduction
---
## 1.1 Base server classes
---
There are four basic concrete server classes:
```python
class socketserver.TCPServer(server_address, RequestHandlerClass, bind_and_activate=True)

class socketserver.UDPServer(server_address, RequestHandlerClass, bind_and_activate=True)

class socketserver.UnixStreamServer(server_address, RequestHandlerClass, bind_and_activate=True)

class socketserver.UnixDatagramServer(server_address, RequestHandlerClass, bind_and_activate=True)
```
这四个类在处理请求时都是同步的，如果想要实现异步还需要`ForkingMixIn` or `ThreadingMixIn`两个类。

## 1.2 创建网络服务的常用步骤
---
1. 创建request handler类，并继承`BaseRequestHandler`,并重写`handler()`方法。

```python
class MyHandler(BaseRequestHandler):
    def handler(self):
        # 处理请求
        pass
```
2. 创建一个server类实例对象，如

```python
server = socketserver.TCPServer(server_address, MyHandler)
```
3. 启动服务(建议使用`with`表达式，这样不用进行最后一步的close方法)

```python
server.serve_forever()
# or
server.handle_request()
```
4. 关闭socket

```python
server.server_close()
```

## 1.3 daemon_threads
---


# 2 Server Creation Notes
---
创建多线程和多进程的基类
```python
class socketserver.ForkingMinIn # available on POSIX

class socketserver.ThreadingMixIn
```
socketserver中预定义的(实现多进程，多线程)server类有：
```python
class socketserver.ForkingTCPServer

class socketserver.ForkingUDPServer

class socketserver.ThreadingTCPServer

class socketserver.ThreadingUDPServer
# 实现方式为：
# class socketserver.ThreadingUDPServer(ThreadingMinIn, UDPServer):
#    pass
```

# 2.1 notes
---
1. 四个基本服务类
    - TCPServer
    - UDPServer
    - UnixStreamServer
    - UnixDatagramServer
2. 两个实现进程和线程的类
    - ForkingMixIn
    - ThreadingMixIn
3. 两个handler父类
    - StreamRequestHandler # 用于实现tcp服务的handler
    - DatagramRequestHandler # 用于实现udp服务的handler
4. 创建**多进程服务**要考虑进程间的**信息交换**，创建**多线程服务**要考虑**锁**的使用。
5. 实现多任务服务器的方法有：多进程，多线程，IO多路复用，协程


# 3 Server Objects
---
```python
class socketserver.BaseServere(server_address, RequestHandlerClass)
```
这个是四个基本服务类的超类。在这个类中定义了一些通用接口，但是大部分方法都没有实现，需要在继承的子类中重写，定义的方法有：
```python
fileno() # 返回socket的文件描述符

handle_request() # 

serve_forever(poll_interval=0.5) # 开始处理请求，直到接收到shutdown() 请求。

service_actions()

shutdown()

server_close()

address_family # AF_INET or AF_UNIX

RequestHandlerClass # 用户提供的handler类，并对每一个request创建一个这个hander类的实例

server_address # 

socket # 

## 定义的类变量
allow_reuse_address # default False

request_queue_size

socket_type # SOCK_STREAM or SOCK_DGRAM

timeout # 用于handler_request()

## 能被重写的方法
finish_request(request, client_address) # 调用RequestHandlerClass实例的handle()

get_request() # return (socket, addr)

handle_error(request, client_address) # 当RequestHandlerClass实例的handle()出现异常

handle_timeout() # 当timeout属性有值

process_request(request, client_address) # 调用finish_request()创建一个RequestHandlerClass的实例

server_activate() # tcp执行listen()时调用

server_bind()

verfy_request(request, client_address) # 必须返回布尔值，用于判断是否处理request
```

# 4 Request Handler Objects
---
## 4.1 Introduction
```python
class socketserver.BaseRequestHandler
```
该类是所有handler类的超类，继承的子类中必须重写`handle()`方法，也可以重写其他方法。**每一个request都会创建一个子类实例**。
```python
setup() # handle()方法执行前被调用，用于执行一些初始化操作。

handle() # 处理所有的具体请求操作。可以使用的实例属性有self.request, self.client_address, self.server

finish() # handle()方法执行后调用的方法
```
## 4.2 subclass
---
```python
class socketserver.StreamRequestHandler

class socketserver.DatagramRequestHandler
```
- 这两个类都是`BaseRequestHanler`的子类
- 重写了父类的`setup()`和`finish()`方法
- 提供了`self.rfile`和`self.wfile`实例属性，用于读写数据
- 两个类的`rfile`和`wfile`属性，支持基本BufferedIOBase接口

# 5 Samples
---
这里直接贴上官方文档的案例。
## 5.1 TCP
---

In [ ]:
# in server
import socketserver

class MyTCPHandler(socketserver.BaseRequestHandler):
    """
    The request handler class for our server.

    It is instantiated once per connection to the server, and must
    override the handle() method to implement communication to the
    client.
    """

    def handle(self):
        # self.request is the TCP socket connected to the client
        self.data = self.request.recv(1024).strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # just send back the same data, but upper-cased
        self.request.sendall(self.data.upper())

if __name__ == "__main__":
    HOST, PORT = "localhost", 9999

    # Create the server, binding to localhost on port 9999
    with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
        # Activate the server; this will keep running until you
        # interrupt the program with Ctrl-C
        server.serve_forever()

也可以使用self.rfile和wfile

In [ ]:
class MyTCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        # self.rfile is a file-like object created by the handler;
        # we can now use e.g. readline() instead of raw recv() calls
        self.data = self.rfile.readline().strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # Likewise, self.wfile is a file-like object used to write back
        # to the client
        self.wfile.write(self.data.upper())

两者不同的地方在于，self.rfile.readline()会调用`recv()`多次，直到遇到换行符。而直接用`recv()`会接收全部数据。

In [ ]:
# in client
import socket
import sys

HOST, PORT = "localhost", 9999
data = " ".join(sys.argv[1:])

# Create a socket (SOCK_STREAM means a TCP socket)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    # Connect to server and send data
    sock.connect((HOST, PORT))
    sock.sendall(bytes(data + "\n", "utf-8"))

    # Receive data from the server and shut down
    received = str(sock.recv(1024), "utf-8")

print("Sent:     {}".format(data))
print("Received: {}".format(received))

Server的输出应该为：

```python
>>> python TCPServer.py
127.0.0.1 wrote:
b'hello world with TCP'
127.0.0.1 wrote:
b'python is nice'

```

Client的输出为：

```python
>>> python TCPClient.py hello world with TCP
Sent:     hello world with TCP
Received: HELLO WORLD WITH TCP

>>> python TCPClient.py python is nice
Sent:     python is nice
Received: PYTHON IS NICE
```

## 5.2 UDP
---


In [ ]:
# in Server
import socketserver

class MyUDPHandler(socketserver.BaseRequestHandler):
    """
    This class works similar to the TCP handler class, except that
    self.request consists of a pair of data and client socket, and since
    there is no connection the client address must be given explicitly
    when sending data back via sendto().
    """

    def handle(self):
        data = self.request[0].strip()
        socket = self.request[1]
        print("{} wrote:".format(self.client_address[0]))
        print(data)
        socket.sendto(data.upper(), self.client_address)

if __name__ == "__main__":
    HOST, PORT = "localhost", 9999
    with socketserver.UDPServer((HOST, PORT), MyUDPHandler) as server:
        server.serve_forever()

In [ ]:
# in Client
import socket
import sys

HOST, PORT = "localhost", 9999
data = " ".join(sys.argv[1:])

# SOCK_DGRAM is the socket type to use for UDP sockets
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# As you can see, there is no connect() call; UDP has no connections.
# Instead, data is directly sent to the recipient via sendto().
sock.sendto(bytes(data + "\n", "utf-8"), (HOST, PORT))
received = str(sock.recv(1024), "utf-8")

print("Sent:     {}".format(data))
print("Received: {}".format(received))

## 5.3 多线程
---
该案例创建Thread-1用于开启服务，并把Thread-1设为守护线程，如果主线程退出，则Thread-1退出。

In [ ]:
import socket
import threading
import socketserver

class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):

    def handle(self):
        data = str(self.request.recv(1024), 'ascii')
        cur_thread = threading.current_thread()
        response = bytes("{}: {}".format(cur_thread.name, data), 'ascii')
        self.request.sendall(response)

class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        sock.sendall(bytes(message, 'ascii'))
        response = str(sock.recv(1024), 'ascii')
        print("Received: {}".format(response))

if __name__ == "__main__":
    # Port 0 means to select an arbitrary unused port
    HOST, PORT = "localhost", 0

    server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)
    with server:
        ip, port = server.server_address

        # Start a thread with the server -- that thread will then start one
        # more thread for each request
        server_thread = threading.Thread(target=server.serve_forever)
        # Exit the server thread when the main thread terminates
        server_thread.daemon = True
        server_thread.start()
        print("Server loop running in thread:", server_thread.name)

        client(ip, port, "Hello World 1")
        client(ip, port, "Hello World 2")
        client(ip, port, "Hello World 3")

        server.shutdown()